In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import numpy as np
import logging
import os
import matplotlib.pyplot as plt
import pickle
from transformers import ViTFeatureExtractor, ViTModel
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from scipy.special import softmax

IMG_SHAPE = 224

import torch
import clip
print(torch.__version__)
print("torch detects GPU: " , torch.cuda.is_available())

1.12.1+cu116
torch detects GPU:  True


# Resizing the images

In [2]:
# load every image in the food folder and resize them to 224x224, then save them in a new folder

# for filename in os.listdir('food'):
#     img = Image.open('food/' + filename)
#     img = img.resize((IMG_SHAPE, IMG_SHAPE))
#     img.save('food_'+str(IMG_SHAPE)+'/' + filename)


# Extracting the features

In [3]:
image = Image.open('food_'+str(IMG_SHAPE)+'/00007.jpg')

feature_extractor = AutoFeatureExtractor.from_pretrained("nateraw/food")
model = AutoModelForImageClassification.from_pretrained("nateraw/food")

# feature_extractor = AutoFeatureExtractor.from_pretrained("aspis/swin-finetuned-food101")
# model = AutoModelForImageClassification.from_pretrained("aspis/swin-finetuned-food101")

#put the models on the GPU
# feature_extractor = feature_extractor.to('cuda')
model = model.to('cuda')
# print(model)

#remove the classifier layer
model.classifier = torch.nn.Identity()
# model.vit.layernorm = torch.nn.Identity()
# print(model)

with torch.no_grad():
    np.set_printoptions(suppress=True)
    inputs = feature_extractor(images=image, return_tensors="pt").to('cuda')
    # print(inputs)
    outputs = model(**inputs)
    # soft = outputs.logits.softmax(dim=-1).cpu().numpy()
    # print(np.argmax(soft))
    # print(outputs)
    print(outputs.logits.cpu().numpy().shape)
    print(outputs.logits.cpu().numpy())



(1, 768)
[[-0.10426077  0.44030023 -0.1440728   0.12320178 -0.44644326 -0.47156906
  -0.45180535  0.07058506 -0.22072954 -0.09709463 -0.06216252 -0.2723686
  -0.18213509 -0.05273443  0.14682387  0.06994285 -0.40889382 -0.09038056
  -0.781425   -0.10016675 -0.07058563  0.29213798  0.0520875   0.17517304
  -0.14301005 -0.16381775 -0.56458443 -0.34005922 -0.19395334 -0.02547447
   0.31792042 -0.13654497  0.4020427  -0.4476825  -0.11272111  0.38092712
   0.33007118  0.28167304 -0.08224933  0.40960187 -0.19431005 -0.1511692
   0.3583383   0.3094159   0.35343406 -0.15485086  0.25468796  0.05406169
   0.37263334  0.12589905 -0.16809288  0.09180164  0.34528515  0.25447512
  -0.3214679   0.10554986  0.4255984   0.36435404 -0.43047392 -0.28916195
   0.10908055 -0.05340453  0.36339182  0.2903532  -0.18700448  0.17492521
   0.6009009   0.28825283  0.09803517  0.19631304  0.35183388  0.17323546
   0.1877055   0.12144285  0.6686396  -0.43249846  0.4111275   0.30453864
  -0.09820481  0.21883997 -0.09

In [4]:
# dictionary with all the image names and their features
features = {}

# convert every image into features
line_id = 0
for filename in os.listdir('food_'+str(IMG_SHAPE)+''):
    id = int(filename.replace(".jpg", ""))
    # print(id)
    line_id += 1
    image = Image.open('food_'+str(IMG_SHAPE)+'/' + filename)
    with torch.no_grad():
        inputs = feature_extractor(images=image, return_tensors="pt").to('cuda')
        feature = model(**inputs).logits
        feature = np.array(feature.cpu().numpy(), dtype=np.float32)
        feature = feature.reshape(-1)
        features[filename.replace(".jpg","")] = feature
        # print(feature[0])
        if line_id % 100 == 0:
            print(line_id)
            # break
        # if id == 5001:
        #     break

# save the features dictionary to a file using pickle
with open('features_vit.pickle', 'wb') as handle:
# with open('features_swin.pickle', 'wb') as handle:
    pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [5]:

# features = {}
# with open('features_mn.pickle', 'rb') as handle:
#     features = pickle.load(handle)
    
print(features['00000'].shape)
print(features['00000'])
print(np.array(features['00000'][0], dtype=np.float32))
print(np.array(features['00001'][0], dtype=np.float32))
print(np.array(features['00002'][0], dtype=np.float32))
print(len(features))

(768,)
[ 0.18819445  0.24721089 -0.12842315 -0.52587193  0.06406281 -0.4308976
 -0.0838914   0.04874071 -0.21432573 -0.15596983 -0.2769205  -0.46969047
 -0.25833338  0.6857904  -0.3082754  -0.02430676  0.5009314  -0.07650276
 -0.05459001 -0.24952707 -0.37622496 -0.06124376  0.20715326  0.5635838
 -0.3806566   0.3973664  -0.4425645   0.67583275  0.35968465 -0.08419175
 -0.5592999  -0.29840055  0.04874619  0.28226915 -0.10060395  0.58187443
 -0.37720725  0.2574794   0.06306023 -0.156716    0.49522996 -0.29469693
 -0.71128345 -0.6370782   0.6165179   0.23061572  0.01115087 -0.30158883
 -0.126507   -0.19405697 -0.69532937  0.26017314  0.5728616  -0.09791184
 -0.06071559 -0.20442076 -0.6070697   0.08207108  0.5108944  -0.36463833
 -0.29296085 -0.45803636 -0.19005887  0.04648723 -0.09337799  0.09145557
 -0.1904791  -0.09047826  0.02107572  0.5237615   0.4858817  -0.17835027
 -0.16027404  0.0052903   0.16596214 -0.6172152   0.2195208  -0.13831858
  0.15614024  0.6327359   0.37706685 -0.441493

In [6]:
# features = {}

# with open('features_vit2.pickle', 'rb') as handle:
#     features = pickle.load(handle)
#     for key in features:
#         features[key] = softmax(features[key])
#     # print(np.array(features['00000'], dtype=np.float32))
#     # print(np.array(features['00001'][0], dtype=np.float32))
#     # print(np.array(features['00002'][0], dtype=np.float32))


# with open('features_vit3.pickle', 'wb') as handle:
#     pickle.dump(features, handle, protocol=pickle.HIGHEST_PROTOCOL)